# Data Filtering 

## Author(s)

Ben Smith

## Existing Notebooks

[Clouds and Data Filtering](https://github.com/ICESAT-2HackWeek/Clouds_and_data_filtering/blob/master/notebooks/Clouds_and_data_quality_rendered.ipynb)

## Learning Outcomes

* Understanding how clouds affect laser-altimetry signals

* Recognizing how these effects are manifest in the ATL06 product

* Gaining familiarity with the ATL06 parameters that can identify cloudy 
returns This part of the tutorial will focus on clouds that cause gross 
errors in surface-height estimates.

### Notes

